In [1]:
import sympy as sp
from sympy import diff, Symbol,symbols
from sympy.physics.mechanics import dynamicsymbols
from sympy.physics.vector import init_vprinting

init_vprinting(use_latex="mathjax", pretty_print=False)

In [2]:
x_M, x_m, x_d, y_d = symbols('x_M x_m x_d y_d')
x, y = dynamicsymbols('x y')
x, y,x_M, x_m, x_d, y_d

(x, y, x_M, x_m, x_d, y_d)

In [3]:
#Let the system state transformation equation be the following:
ste = y - sp.tan((sp.pi/(x_M - x_m))*(x-x_m) - (sp.pi/2))

In [4]:
#Let the desired state transformation equation be the following:
ste_d = y_d - sp.tan((sp.pi/(x_M - x_m))*(x_d-x_m) - (sp.pi/2))

In [5]:
ste, ste_d

(y + cot(pi*(-x_m + x)/(x_M - x_m)), y_d + cot(pi*(x_d - x_m)/(x_M - x_m)))

In [6]:
#Considering y → yd, when x → xd, and xm < x < xM for
#all y ∈ R. Thus, by choosing a suitable function, the state
#transition function can convert an unbounded state y to a
#bounded state x, and then we can obtain the following:

In [19]:
x_state = sp.solve([ste],[x])[x]
x_state.simplify()

(-x_M*acot(y) + x_m*acot(y) + pi*x_m)/pi

In [20]:
xd_state = sp.diff(x_state,'t',1)
xd_state.simplify()

(x_M - x_m)*y'/(pi*(y**2 + 1))

In [21]:
xdd_state = sp.diff(x_state,'t',2)
xdd_state.simplify()

(-2*x_M*y*y'**2 + 2*x_m*y*y'**2 + (x_M - x_m)*(y**2 + 1)*y'')/(pi*(y**2 + 1)**2)

In [22]:
# permanent magnet linear motor (PMLM) model
R_m, K_f, K_e, B, F_fric, F_ripple= symbols('R_m K_f K_e B F_fric F_ripple')

M = R_m/K_f
F = F_fric + F_ripple

u = M*xdd_state + (K_e + B)*xd_state + F
u

F_fric + F_ripple + (B + K_e)*(x_M*y'/(y**2 + 1) - x_m*y'/(y**2 + 1))/pi + R_m*(x_M*y'' - 2*x_M*y*y'**2/(y**2 + 1) - x_m*y'' + 2*x_m*y*y'**2/(y**2 + 1))/(pi*K_f*(y**2 + 1))

In [25]:
#simplifing into a general form of the dynamics model
coeffs = sp.Matrix([y.diff('t',1), y.diff('t',2)])
expr = sp.collect(sp.expand(u), syms=coeffs[:])
expr

F_fric + F_ripple + (R_m*x_M/(pi*K_f*y**2 + pi*K_f) - R_m*x_m/(pi*K_f*y**2 + pi*K_f))*y'' + (-2*R_m*x_M*y/(pi*K_f*y**4 + 2*pi*K_f*y**2 + pi*K_f) + 2*R_m*x_m*y/(pi*K_f*y**4 + 2*pi*K_f*y**2 + pi*K_f))*y'**2 + (B*x_M/(pi*y**2 + pi) - B*x_m/(pi*y**2 + pi) + K_e*x_M/(pi*y**2 + pi) - K_e*x_m/(pi*y**2 + pi))*y'

In [26]:
# NEW Dynamics after state transformation

#mass term M
M_y = expr.coeff(y.diff('t',2))
M_y.simplify() 

R_m*(x_M - x_m)/(pi*K_f*(y**2 + 1))

In [27]:
#the Koch force/centrifugal force term C
C_y = expr.coeff(y.diff('t',1))
C_y.simplify() 

(B*x_M - B*x_m + K_e*x_M - K_e*x_m)/(pi*(y**2 + 1))

In [32]:
# the friction F of the linear motor mechanical system after state transition
F_y = expr - (M_y*y.diff('t',2)) - (C_y*y.diff('t',1))
F_y

F_fric + F_ripple + (-2*R_m*x_M*y/(pi*K_f*y**4 + 2*pi*K_f*y**2 + pi*K_f) + 2*R_m*x_m*y/(pi*K_f*y**4 + 2*pi*K_f*y**2 + pi*K_f))*y'**2